In [316]:
import pandas as pd
import numpy as np
import matplotlib as mplt
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy

import time
from sklearn.decomposition import PCA

from sklearn.neighbors import NearestNeighbors
from sklearn import neighbors

from matplotlib.colors import ListedColormap

import statsmodels.api as sm
from sklearn import linear_model



In [ ]:
data = pd.DataFrame.from_csv("/Users/raphaelpresberg/Desktop/Stevens/BIA 652 Multivariate Data Analysis/NYC_Taxis/NYC_taxis.csv")


#data understanding
data.head()
data.tail(10)
sLength = len(data['Pickup Time'])
data['Trip time'] = pd.Series(np.random.randn(sLength), index = data.index)
data['new_col'] = range(0, sLength)
data = data.reset_index()
data['new_col'] = data.index
print(data.head())
data.info()
data.describe()



In [ ]:

def basics_info (initial_numpy_array):
    print("Mean: ", np.mean(initial_numpy_array))
    #median cost of a trip without @ without taxes
    print("Median: ", np.median(initial_numpy_array))
    #The numpy.ptp() function returns the range (maximum-minimum) of values along an axis.
    print("Range: ", np.ptp(initial_numpy_array))
    #standard deviation & variance
    print("SD: ", np.std(initial_numpy_array))
    print("Variance: ", np.var(initial_numpy_array))
    plt.plot(initial_numpy_array)
    plt.show() 


#removing outliers and compute the distribution plot

def removeOutliers(x):
    a = np.array(x)
    outlierConstant = 1
    upper_quartile = np.percentile(a, 75)
    lower_quartile = np.percentile(a, 25)
    IQR = (upper_quartile - lower_quartile) * outlierConstant
    quartileSet = (lower_quartile - IQR, upper_quartile + IQR)
    resultList = []
    for y in a.tolist():
        if y >= quartileSet[0] and y <= quartileSet[1]:
            resultList.append(y)
    return resultList

def prob_plot(variable):
    
    mu = np.mean(variable)
    sigma = np.std(variable)

    mu, sigma = mu, sigma
    x = mu + sigma * np.array(variable)

    # the histogram of the data
    n, bins, patches = plt.hist(x, 50, normed=1, facecolor='g', alpha=0.95)


    plt.xlabel("Prob ")
    plt.ylabel('Probability')
    plt.title('Proba Plot')
   # plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
    plt.axis([20, 190, 0, 0.03])
    plt.grid(True)
    plt.show()
    



In [ ]:
#*******************************************
#STUDY FOR TOTAL AMOUNT 
#*******************************************
total_amount = data.as_matrix(columns = data.columns[19:20])

basics_info(total_amount)
good_total_amount = removeOutliers(total_amount)
prob_plot(good_total_amount)


In [ ]:
#*******************************************
#STUDY FOR FARE AMOUNT 
#*******************************************

fare_amount = data.as_matrix(columns=data.columns[17:118])

basics_info(fare_amount)
good_fair_amount = removeOutliers(data[['Fare Amount']])
prob_plot(good_fair_amount)


In [ ]:
#*******************************************
#STUDY FOR TIP AMOUNT 
#*******************************************

tip_amount = data.as_matrix(columns=data.columns[14:15])

basics_info(tip_amount)
good_tip_amount = removeOutliers(tip_amount)
prob_plot(good_tip_amount)


In [ ]:
x = total_amount
pca = PCA(n_components=1)
pca.fit(x)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)  



In [ ]:
#a = pickup time
#b = dropoff time
a = data.as_matrix(columns=data.columns[0:1])
b = data.as_matrix(columns=data.columns[2:3])
c = "0:00:00.000"
a
#x = a[0]
c.split(':')


In [ ]:
#%H # ==> heure pour heures de 1 a 23
#%M # ==> minutes 

# string to time tuple
time_str = "17:53:59"
time_tuple = time.strptime(time_str, "%H:%M:%S")
print(time_tuple)
x= time_tuple[3:5]
nb_minutes = x[0]*60 + x[1]
nb_minutes

In [ ]:
test= pd.DataFrame(a)[1:100]
new_dt = test.apply(lambda x: str(x).split(":"), axis=1)
new_dt
#hours = new_dt.as_matrix()
#hours


In [ ]:
x = np.array(["fdp:connars",  "fdp:salopr",  "ju:jkfen"])

x.rpartition(x,':')


In [ ]:
type(data.columns[1])


In [ ]:
#if (drop off date = pickup date):
#    Dropoff Time - Pickup Time
#else: 
#    abs_value =abs(DT - PT)
#    1440 - abs_value

data.head()



In [338]:
list(data)

['Pickup Date',
 'Pickup Time',
 'Dropoff Date',
 'Dropoff Time',
 'Vendor ID',
 'Passenger Count',
 'Trip Distance (in miles)',
 'Pickup Longitude',
 'Pickup Latitude',
 'Rate Code',
 'Store and Forward Flag',
 'Dropoff Longitude',
 'Dropoff Latitude',
 'Payment Type',
 'Fare Amount',
 'Surcharge',
 'MTA Tax',
 'Tip Amount',
 'Tolls Amount',
 'Total Amount',
 'Trip time',
 'new_col']

In [ ]:
#Tip Analysis 
#Question: generous/not generous customer

#variable to use ==> total amount, passenger count, trip distance, rate code, surcharge
#Output ==> tip amount

tip_data = data[['Total Amount', 'Passenger Count', 'Trip Distance (in miles)', 'Rate Code', 'Surcharge', 'Tip Amount']]
tip_data
tip_data.describe()

#Creating train and test dataset (2/3 - 1/3)
msk = np.random.rand(len(tip_data)) < 0.66
tip_train = tip_data[msk]
tip_test = tip_data[~msk]


#2 categories to classify ==> Not Generous, Generous

#tip_data["Generous or not"] = 0
tip_data.info()
tip_data.head()
tip_data.tail()
#neigh = KNeighborsClassifier(n_neighbors=3)
#neigh.fit(X, y) 
#KNeighborsClassifier(...)
#print(neigh.predict([[1.1]]))

#print(neigh.predict_proba([[0.9]]))



In [ ]:
a = tip_data.as_matrix(columns = tip_data.columns[5:6])

tip_data["Generous ?"] = ""

i=0
for i in range(0,727309) :
    if a[i]== 0:
        tip_data.ix[[i],["Generous ?"]] = "not generous"
    else:
        tip_data.ix[[i],["Generous ?"]] = "generous"
        
tip_data

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [ ]:
tip_data["Generous ?"] = ""

tip_data["Generous ?"]

In [334]:
#https://towardsdatascience.com/simple-and-multiple-linear-regression-in-python-c928425168f9

#***************************************************************************************************************
#Multiple Regression 
#***************************************************************************************************************


# WITH STATSMODELS

target = pd.DataFrame(data[['Tip Amount']])

df = pd.DataFrame( data[['Total Amount', 'Passenger Count', 'Trip Distance (in miles)', 'Rate Code', 'Surcharge']])

#Multiple Regression 
#Prediction of the tip the customer will give in function of the other variables we selected 
X = df[['Total Amount', 'Passenger Count', 'Trip Distance (in miles)', 'Rate Code', 'Surcharge']]
y = target["Tip Amount"]

model = sm.OLS(y, X).fit()
predictions = model.predict(X) 

model.summary()

#Analysis: R^2 = 0.451 means this model explain only 45,1% of the variance in our dependent variable


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Tip Amount   R-squared:                       0.451
Model:                            OLS   Adj. R-squared:                  0.451
Method:                 Least Squares   F-statistic:                 1.194e+05
Date:                Tue, 10 Apr 2018   Prob (F-statistic):               0.00
Time:                        17:23:14   Log-Likelihood:            -1.5591e+06
No. Observations:              727310   AIC:                         3.118e+06
Df Residuals:                  727305   BIC:                         3.118e+06
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Total Amount                 0.1039      0.000    571.609      0.000       0.104       0.104
Passenger Count             -0.0502      0.002    -32.171      0.000      -0.053      -0.047
Trip Distance (in miles) -3.052e-08   1.72e-07     -0.177      0.859   -3.68e-07    3.07e-07
Rate Code                   -0.1811      0.003    -54.792      0.000      -0.188      -0.175
Surcharge                    0.0402      0.006      6.227      0.000       0.028       0.053
==============================================================================
Omnibus:                  2575718.182   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):   20341246372818.117
Skew:                          68.946   Prob(JB):                         0.00
Kurtosis:                   25910.677   Cond. No.                     3.79e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.79e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [331]:
# WITH SCICKIT

target = pd.DataFrame(data[['Tip Amount']])

df = pd.DataFrame( data[['Total Amount', 'Passenger Count', 'Trip Distance (in miles)', 'Rate Code', 'Surcharge']])

#Multiple Regression 
#Prediction of the tip the customer will give in function of the other variables we selected 
#X = df
X = df[['Total Amount', 'Passenger Count', 'Trip Distance (in miles)', 'Rate Code', 'Surcharge']]

y = target["Tip Amount"]

test_prediction = pd.DataFrame([23, 3, 2.4, 0.5, 0])

lm = linear_model.LinearRegression()
model = lm.fit(X,y)

predictions = lm.predict(X)

print("prediction", predictions)

# percentage of explained variance of the predictions
print("R^2 (percentage of variance): " ,lm.score(X,y))

#coefficients for the predictors
print("coef: ", lm.coef_)

# intercept for the predictors
print("intercept", lm.intercept_)




prediction [ 0.82667978  0.60626854  1.14334155 ...,  0.82603476  1.19394272
  6.87608117]
R^2 (percentage of variance):  0.304141587458
coef:  [  1.05116571e-01  -3.10456909e-02  -2.46506861e-08  -1.31452592e-01
   1.03826548e-01]
intercept -0.146080857344


In [336]:
#***************************************************************************************************************
#Classification ==> KNN Algo
#***************************************************************************************************************
n_neighbors = 15

X = df[['Total Amount', 'Passenger Count', 'Trip Distance (in miles)', 'Surcharge']]
y = target["Tip Amount"]


# Create color maps
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])



for weights in ['uniform', 'distance']:
    # we create an instance of Neighbours Classifier and fit the data.
    clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
    clf.fit(X, y)

    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure()
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

    # Plot also the training points
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold,
                edgecolor='k', s=20)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title("3-Class classification (k = %i, weights = '%s')"
              % (n_neighbors, weights))

plt.show()







TypeError: unhashable type: 'slice'

In [ ]:
i=0  
if tip_data.ix[[i],["Tip Amount"]]  == 0:
    tip_data.ix[[i],["Generous or not"]] == "not generous"

In [ ]:
#tip_data.ix[[0,1,2], ['Total Amount']]


In [ ]:
grouper = tip_data.groupby("Tip Amount")
grouper

In [ ]:
tip_train.groupby("Total Amount").Surcharge.describe()

In [ ]:
i = 0